In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pytesseract

import subprocess
from pathlib import Path

from maddrive_adas.sign_det.base import AbstractSignClassifier
from maddrive_adas.sign_det.classifier import EncoderBasedClassifier
from maddrive_adas.utils.fs import imread_rgb

try:
    output = subprocess.check_output(
        'tesseract -v',
        stderr=subprocess.STDOUT
    ).decode()
    if 'tesseract' not in output:
        raise subprocess.CalledProcessError
except subprocess.CalledProcessError:
    print('Unable to call tessecact. Install and add tesseract to PATH variable.')
    print('Link: https://tesseract-ocr.github.io/tessdoc/Downloads.html')
    raise subprocess.CalledProcessError

def crop_img(img, xscale=1.0, yscale=1.0):
    center_x, center_y = img.shape[1] / 2, img.shape[0] / 2
    width_scaled, height_scaled = img.shape[1] * xscale, img.shape[0] * yscale
    left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
    top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
    img_cropped = img[int(top_y):int(bottom_y), int(left_x):int(right_x)]
    return img_cropped

PROJECT_ROOT = Path('.')
data_path: Path = Path(PROJECT_ROOT / 'SignDetectorAndClassifier/data/additional_sign')

TARGET_WIDTH = 40
erode_kernel = np.ones((2, 2), np.uint8)

for file in data_path.iterdir():
    if '3.24' in str(file):  # or '3.25' in str(file):
        print(file)
        # plt.figure()
        # f, axarr = plt.subplots(7,1, figsize=(30, 15)) 
        
        img = imread_rgb(file)
        # axarr[0].imshow(img)
        
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        # axarr[1].imshow(img, cmap='gray')
        
        img = crop_img(img, xscale=0.7, yscale=0.4)
        # axarr[2].imshow(img, cmap='gray')
        
        scale_x = TARGET_WIDTH / img.shape[0]
        img = cv2.resize(img, (int(img.shape[0] * scale_x), TARGET_WIDTH), interpolation=cv2.INTER_AREA)
        # axarr[3].imshow(img, cmap='gray')

        img = cv2.GaussianBlur(img, (7, 7), 0)
        # axarr[4].imshow(img, cmap='gray')
        img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 5,5)
        
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
        img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel, iterations=2)
        # axarr[5].imshow(opening, cmap='gray')        
        
        img = cv2.erode(img, erode_kernel, cv2.BORDER_CONSTANT) 
        img = cv2.dilate(img, erode_kernel, cv2.BORDER_CONSTANT,  iterations=1) 
        # res = c.classify(img)
        # c._fixup_signs_with_text(img, ('3.25', 1))
        plt.imshow(img, cmap='gray')
        print(pytesseract.image_to_string(img, config='--psm 13').split('\n\n')[0])
        plt.show()
        input()
assert False, 'Check windows'

In [ ]:
img.shape

In [ ]:
list(int(scale / x) for x in img.shape)